# Scatterplots
## Via Holoviews


### Preamble:

In [ ]:
import numpy as np
import pandas as pd
import whattheplot as wtp

In [ ]:
import holoviews as hv
hv.extension('bokeh')
renderer = hv.renderer('bokeh')
renderer.size = 150

### Data

In [ ]:
df = wtp.data.load_iris()
df.drop(columns='species_code', inplace=True)
df.head()

## Scatter Plots, declarative style
### Graph-elements, not pixel-maps
First things first; a basic scatterplot transform

In [ ]:
points = hv.Points(df, kdims=['sepal_length', 'petal_width']).options(size=5)
points

Note, we can change properties on the fly; this is data-with-representation, not a pixel-map:

In [ ]:
points.options(size=15, color='orange')

How does that work? By attaching the data to the plot element...

In [ ]:
points.data.head()

### Declaring data-properties 
(AKA fold-by / group-by / select-by handles)

In [ ]:
dataset = hv.Dataset(df, kdims='species')

We can map over the 'kdims', to create a sort of dict-of-plots - a 'holomap'.

In [ ]:
%%opts Points (size=15)
points_holomap =dataset.to(hv.Points, kdims=['sepal_length', 'petal_width'])
points_holomap.data

In [ ]:
points_holomap

In [ ]:
%output size=200

In [ ]:
%%opts Points [tools=['hover']] (size=10 alpha=.8 muted_alpha=0.05)

hv.NdOverlay(points_holomap).redim.range(
    sepal_length=(4.,8.),
    petal_width=(0.,3.)
)

In [ ]:
dataset.vdims

In [ ]:
import itertools
pairings = list(itertools.combinations(dataset.vdims, 2))
pairings

In [ ]:
all_the_things = [
    hv.NdOverlay(
        dataset.to(hv.Points, kdims=list(p))
    ).relabel(f'{p[0]} vs {p[1]}') for p in pairings
]

In [ ]:
%output size=100

In [ ]:
%%opts Points [tools=['hover'] fontsize={'legend':8} show_legend=False ] (size=6 alpha=.8)
layout=hv.Layout(all_the_things).cols(3)
layout

In [ ]:
renderer.save(layout, 'all_the_iris')